**Author**: Ismaele Gorgoglione

## Hashing

In [1]:
## Define some function useful for testing
import random

## generate an array of n random integers up to b
def get_random_array(n, b = 50):
    return [random.randint(0, b) for _ in range(n)]

---

### Open Addressing with linear probing

[Open addressing](https://en.wikipedia.org/wiki/Open_addressing) is a collision resolution technique used for handling collisions in hashing. 

All the items are stored in a table of size $\alpha n$, where $n$ is the number of keys and $\alpha > 1$ is the load factor.

Initially, the table contains only a special value ```None``` which says that the entry is empty. Another 
special value, say character ```'D'``` is used to mark a entry that contained a key that has been deleted.

A hash functon $h()$ is used to specify the order of entries to probe for a key to be inserted/searched/deleted. 
We start by probing $h(k)$ and, with linear probing, the sequence of probes $S(k)$ is $h(k), h(k)+1, h(k)+2, \ldots$ , modulo $\alpha n$.

- **Insert** adds the key in the first empty slot that we found with positions in $S(k)$.
- **Lookup** is performed by checking positions in $S(k)$ until we find either the key or ```None```.
- **Delete** is performed by first sesrching the key and then by replacing it with ```'D'```. Why don't we use ```None``` instead? 

### Exercise: Open Addressing with linear probing
Complete the implementation below by implementing ```Lookup``` and ```Delete```.


**Optional:** Try to implement quadratic probing. This is the technique employed by Python's set and dictionary. **Note:** not all the value of $c_1$ and $c_2$ are correct. If you generate them at random, for some of them you'll not be able to insert all the key and you'll get a random loop even if the code is correct. 

In [2]:
class linear_probing_set:
    def __init__(self, size):
        
        self.T = [None]*size
        self.prime = 993319
        self.a = random.randint(2, self.prime-1)
        self.b = random.randint(2, self.prime-1)
        self.n_keys = 0
    
    def insert(self, key): # fix len(T) < self.n_keys if you want
        if self.lookup(key):
            return
        h = self.hash(key)
        while self.T[h] != None and self.T[h] != 'D':
            h += 1
            if h == len(self.T):
                h = 0
        self.T[h] = key
        self.n_keys += 1
    
    # Return True if key is in the set, False otherwise
    def lookup(self, key):
        h = self.hash(key)
        while self.T[h] != None:
            if self.T[h] == key:
                return True
            h += 1
            if h == len(self.T):
                h = 0
        return False

    def delete(self, key):
        h = self.hash(key)
        while self.T[h] != None:
            if self.T[h] == key:
                self.T[h] = 'D'
                self.n_keys -= 1
            h += 1
            if h == len(self.T):
                h == 0
        return False

    def hash(self, key):
        return ((self.a*key + self.b) % self.prime) % len(self.T)
    
    def len(self):
        return self.n_keys

In [3]:
## Implementation test

n = 10000

a = get_random_array(n, n)

queries = get_random_array(n, n)

lp_set = linear_probing_set(2*n)
std_set = set()

for key in a:
    lp_set.insert(key)
    std_set.add(key)

assert len(std_set) == lp_set.len(), "Fail len!"     
    
for key in a:
    assert lp_set.lookup(key) == True, "Lookup fail a"
  
for key in queries:
    assert lp_set.lookup(key) == (key in std_set), "Lookup fail queries"
    
for key in a[:300]:
    lp_set.delete(key)
    try:
        std_set.remove(key)
    except:
        pass # the key has been already removed
          
    assert lp_set.lookup(key) == (key in std_set), "Lookup fail delete"    


In [4]:
%timeit for key in queries: lp_set.lookup(key)
    
%timeit for key in queries: key in std_set

6.85 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
626 µs ± 47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


----
### Hashing with Chains
Instead of just storing the elements in the slots in the table $T$, let every slot be a list which contains all the elements which are in the table and map to that slot. Our operations now become:

- `Insert` $(k)$: hash $k$ to an index $i$ in the table. You may want to check if $k$ is already in the set first.
- `Lookup` $(k)$: search for $k$ in the list by iterating through all the list.
- `Delete` $(k)$: search for $k$ and then remove it from the list.

Lookup and Delete takes $O(s)$ time where $s$ is the size of the list. We define $\alpha = \frac{n}{m}$ as the **load factor**. If we assume simple uniform hashing, then each element has equal probability to go into any slot. So after $n$ independent elements have been inserted we have and expected length of $\frac{n}{m} = \alpha$ for each chain by linearity of expectation. So the run time of all the above operations is time to hash + time to do these operations which is $O(1 + \alpha)$.

### Exercise: Hashing with Chains
Complete the implementation below by implementing ```Lookup``` and ```Delete```.

In [5]:
class chaining_set:
    def __init__(self, size):
        
        self.T = []
        for _ in range(size):
            self.T.append([])
        
        self.prime = 993319
        self.a = random.randint(2, self.prime-1)
        self.b = random.randint(2, self.prime-1)
        self.n_keys = 0
        
    def insert(self, key):
        if self.lookup(key):
            return
        
        h = self.hash(key)
        self.T[h].append( key )
        self.n_keys += 1
    
    # return True if key is in the set, False otherwise
    def lookup(self, key):
        h = self.hash(key)
        for i in range(len(self.T[h])):
            if self.T[h][i] == key:
                return True
        return False
    
    def delete(self, key):
        h = self.hash(key)
        for i in range(len(self.T[h])):
            if self.T[h][i] == key:
                self.T[h][i], self.T[h][-1] = self.T[h][-1], self.T[h][i] # swap the key with the last key in the list
                self.T[h].pop()
                self.n_keys -= 1
                break
            
    def hash(self, key):
        return ((self.a*key + self.b) % self.prime) % len(self.T)
    
    def len(self):
        return self.n_keys

In [6]:
## Implementation test

n = 10000

a = get_random_array(n, n)

queries = get_random_array(n, n)

c_set = chaining_set(2*n)
std_set = set()

for key in a:
    c_set.insert(key)
    std_set.add(key)

assert len(std_set) == c_set.len(), "Fail len!"     
    
for key in a:
    assert c_set.lookup(key) == True, "Lookup fail a"
  
for key in queries:
    assert c_set.lookup(key) == (key in std_set), "Lookup fail queries"
    
for key in a[:300]:
    c_set.delete(key)
    try:
        std_set.remove(key)
    except:
        pass # the key has been already removed
          
    assert c_set.lookup(key) == (key in std_set), "Lookup fail delete"  

In [7]:
%timeit for key in queries: c_set.lookup(key)
    
%timeit for key in queries: key in std_set

9.24 ms ± 452 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
641 µs ± 78.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


----

### Exercise: Dictionary
Modify the previous code (i.e., Hashing with Chains) to implement a dictionary, i.e., store a value together with each key. 
You need to implement methods:
- ```Insert(key, value)```: insert the key with its value. If the key was already present, change its value;
- ```Delete(key)```: remove the key;
- ```Lookup(key)```: return True if the key is present, False otherwise;
- ```Value(key)```: return the value associated with the key. It returns None, if the key is not present.

I suggest to store pairs (key, value) within the lists.


**Optional**. 
Implement ```keys()```, ```values()```, and ```items()``` which allows you to iterate over keys, values, and pairs (key, value) respectively. You have to use ```yield``` to implement each generator.  

In [8]:
class chaining_dict:
    def __init__(self, size):
        
        self.T = []
        for _ in range(size):
            self.T.append([])
        
        self.prime = 993319
        self.a = random.randint(2, self.prime-1)
        self.b = random.randint(2, self.prime-1)
        self.n_keys = 0
        
    def insert(self, key, value): # for each hash we will have a tuple (key, val)
        h = self.hash(key)
        for i, tup in enumerate(self.T[h]):
            if tup[0] == key:
                self.T[h][i] = (key, value)
                return
        self.T[h].append((key, value))
        self.n_keys += 1
    
    def lookup(self, key):
        h = self.hash(key)
        for tup in self.T[h]:
            if tup[0] == key:
                return True
        return False
    
    def delete(self, key):
        h = self.hash(key)
        for i, tup in enumerate(self.T[h]):
            if tup[0] == key:
                self.T[h][i], self.T[h][-1] = self.T[h][-1], self.T[h][i]
                self.T[h].pop()
                self.n_keys -= 1
                break

    def value(self, key):
        h = self.hash(key)
        for tup in self.T[h]:
            if tup[0] == key:
                return tup[1]
        return None

    def hash(self, key):
        return ((self.a*key + self.b) % self.prime) % len(self.T)
    
    def len(self):
        return self.n_keys
    
# OPTIONAL 
    def keys(self):
        for el in self.T:
            if len(el):
                for tup in el:
                    yield tup[0]
                    
    def values(self):
        for el in self.T:
            if len(el):
                for tup in el:
                    yield tup[1]

    def items(self):
        for el in self.T:
            if len(el):
                for tup in el:
                    yield tup[0], tup[1]      

In [9]:
## Implementation test

n = 10000

keys = get_random_array(n, n)
values = get_random_array(n, n)
a = list(zip(keys, values))

c_dic = chaining_dict(2*n)
std_dic = dict()

for key, value in a:
    c_dic.insert(key, value)
    std_dic[key] = value

assert len(std_dic) == c_dic.len(), "Fail len!"     
    
for key in keys:
    assert c_dic.lookup(key) == True, "Lookup fail a"

for key in keys:
    assert c_dic.lookup(key) == (key in std_dic), "Lookup fail queries"
    
for key in keys[:300]:
    c_dic.delete(key)
    try:
        std_dic.pop(key)
    except:
        pass # the key has been already removed
          
    assert c_dic.lookup(key) == (key in std_dic), "Lookup fail delete"  

In [10]:
# Testing the optional part
k = list(c_dic.keys())
v = list(c_dic.values())

assert len(std_dic.keys()) == len(k), "Fail keys len!"
assert len(std_dic.values()) == len(v), "Fail values len!"

assert sorted(k) == sorted(std_dic.keys()), "Fail"
assert sorted(v) == sorted(std_dic.values()), "Fail"


In [11]:
# Running yield (optional part)
import random
chaining = chaining_dict(50)

for item in range(30):
    chaining.insert(item, random.randint(1,10000)) # adding "random" elements

key_test = chaining.keys()
value_test = chaining.values()
item_test = chaining.items()

In [12]:
# Instead of using a for loop, we can also call next() on the generator object. 
# This is useful for testing a generator
next(key_test)

22

In [13]:
# Instead of using a for loop, we can also call next() on the generator object. 
# This is useful for testing a generator
next(value_test)

6303

In [14]:
# Instead of using a for loop, we can also call next() on the generator object. 
# This is useful for testing a generator
next(item_test)

(22, 6303)